Imports

In [33]:
import os
import rasterio
from rasterio.errors import RasterioError
import numpy as np

# Expected files per fire folder
expected_files = {
    "ndvi_pre": "pre_ndvi.tif",
    "ndvi_post": "post_ndvi.tif",
    "nbr_pre": "pre_nbr.tif",
    "nbr_post": "post_nbr.tif",
    "chirps_precip": "chirps_precip.tif",
    "era5_temp": "era5_temp.tif",
    "landcover": "landcover.tif",
    "srtm_dem": "srtm_dem.tif",
    "modis_burned_area": "modis_burned_area.tif"
}


Validate all fire event folders exist and contain files required

In [34]:
def check_expected_files(fire_root, expected_files):
    print("\n# Checking folder structure and file existence #")

    fire_folders = [
        os.path.join(fire_root, f) 
        for f in os.listdir(fire_root) 
        if os.path.isdir(os.path.join(fire_root, f))
    ]

    results = {}

    for folder in fire_folders:
        fire_name = os.path.basename(folder)
        print(f"\nChecking: {fire_name}")

        results[fire_name] = {"missing": [], "present": []}

        for key, filename in expected_files.items():
            path = os.path.join(folder, filename)
            if os.path.exists(path):
                results[fire_name]["present"].append(filename)
            else:
                results[fire_name]["missing"].append(filename)

        if len(results[fire_name]["missing"]) == 0:
            print("     All expected files present")
        else:
            print("     Missing files:")
            for m in results[fire_name]["missing"]:
                print(f"    - {m}")

    return results


Validate raster integrity for each file

In [35]:
def validate_raster(path):
    """Return (is_valid, message, metadata)."""
    try:
        with rasterio.open(path) as src:
            arr = src.read(1)

            if arr.size == 0:
                return False, "Empty raster", None
            
            if np.all(np.isnan(arr)):
                return False, "Raster contains only NaN values", None

            meta = {
                "shape": arr.shape,
                "crs": src.crs,
                "transform": src.transform,
                "dtype": src.dtypes[0],
                "nodata": src.nodata
            }

            return True, "OK", meta

    except RasterioError as e:
        return False, f"Rasterio error: {str(e)}", None
    except Exception as e:
        return False, f"General error: {str(e)}", None


Validate raster alignment in fire folder

In [36]:
def check_alignment(metadata_dict):
    """Check if all rasters share the same resolution, shape, and transform."""
    metas = list(metadata_dict.values())

    shapes = {m["shape"] for m in metas}
    transforms = {m["transform"] for m in metas}

    aligned = (len(shapes) == 1) and (len(transforms) == 1)

    return aligned, shapes, transforms


Run validation on all fires

In [37]:
def validate_all_rasters(fire_root, expected_files):
    print("\n# Validating raster contents #")

    fire_folders = [
        os.path.join(fire_root, f)
        for f in os.listdir(fire_root)
        if os.path.isdir(os.path.join(fire_root, f))
    ]

    for folder in fire_folders:
        fire_name = os.path.basename(folder)
        print(f"\n{fire_name}")

        metadata_per_raster = {}

        for key, filename in expected_files.items():
            path = os.path.join(folder, filename)

            if not os.path.exists(path):
                print(f"Missing: {filename}")
                continue

            valid, message, meta = validate_raster(path)

            if not valid:
                print(f"    {filename} — INVALID: {message}")
            else:
                print(f"    {filename} — OK")
                metadata_per_raster[filename] = meta

        # Alignment check
        if len(metadata_per_raster) > 1:
            aligned, shapes, transforms = check_alignment(metadata_per_raster)

            if aligned:
                print("\n    Rasters aligned (shape + transform match)")
            else:
                print("\n    Rasters NOT aligned:")
                print(f"    Shapes found: {shapes}")
                print(f"    Transforms found: {transforms}")


Run functions

In [39]:
fire_root = "../data/raw"
file_check = check_expected_files(fire_root, expected_files)

validate_all_rasters(fire_root, expected_files)


# Checking folder structure and file existence #

Checking: bootleg_fire
     All expected files present

Checking: caldor_fire
     All expected files present

Checking: camp_fire
     All expected files present

Checking: carr_fire
     All expected files present

Checking: dixie_fire
     All expected files present

Checking: east_troublesome_fire
     All expected files present

Checking: glass_fire
     All expected files present

Checking: red_salmon_fire
     All expected files present

Checking: zogg_fire
     All expected files present

# Validating raster contents #

bootleg_fire
    pre_ndvi.tif — OK
    post_ndvi.tif — OK
    pre_nbr.tif — OK
    post_nbr.tif — OK
    chirps_precip.tif — OK
    era5_temp.tif — OK
    landcover.tif — OK
    srtm_dem.tif — OK
    modis_burned_area.tif — OK

    Rasters NOT aligned:
    Shapes found: {(1861, 3712), (12, 23), (112, 223), (1861, 3711), (57, 112), (1860, 3711)}
    Transforms found: {Affine(0.04491576420597607, 0.0, -121.2276475